In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
import numpy as np
import pandas as pd
from keras import layers

In [ ]:
#parameters
classes = 12
batch_size=128
train_total = 3783
validation_total = 967

In [ ]:
model_inception = InceptionV3(include_top=False, weights = 'imagenet',input_shape=(224,224,3))

In [ ]:
model_inception.summary()

In [ ]:
x = model_inception.get_layer('mixed10').output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=model_inception.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
for layer in model_inception.layers:
    layer.trainable= False

In [ ]:
model.get_layer('batch_normalization_94').trainable= True
model.get_layer('batch_normalization_86').trainable= True
model.get_layer('conv2d_94').trainable= True
model.get_layer('batch_normalization_93').trainable= True
model.get_layer('batch_normalization_92').trainable= True
model.get_layer('batch_normalization_89').trainable= True
model.get_layer('batch_normalization_88').trainable= True
model.get_layer('conv2d_86').trainable= True
model.get_layer('conv2d_94').trainable= True
model.get_layer('conv2d_93').trainable= True
model.get_layer('conv2d_92').trainable= True
model.get_layer('conv2d_89').trainable= True
model.get_layer('conv2d_88').trainable= True
model.get_layer('batch_normalization_91').trainable= True
model.get_layer('batch_normalization_87').trainable= True
model.get_layer('conv2d_91').trainable= True
model.get_layer('conv2d_87').trainable= True
model.get_layer('batch_normalization_90').trainable= True
model.get_layer('conv2d_90').trainable= True

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-7),
             metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath='inception_fixed_bottom_save_best.hdf5', verbose=1, save_best_only=True, save_weights_only=True)


In [ ]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=[checkpointer])

Epoch 1/50
Epoch 00000: val_loss improved from inf to 0.44380, saving model to inception_fixed_bottom_save_best.hdf5
346s - loss: 0.6023 - acc: 0.7946 - val_loss: 0.4438 - val_acc: 0.8538
Epoch 2/50
Epoch 00001: val_loss did not improve
201s - loss: 0.5979 - acc: 0.7952 - val_loss: 0.4597 - val_acc: 0.8570
Epoch 3/50
Epoch 00002: val_loss improved from 0.44380 to 0.44031, saving model to inception_fixed_bottom_save_best.hdf5
235s - loss: 0.5922 - acc: 0.7971 - val_loss: 0.4403 - val_acc: 0.8538
Epoch 4/50
Epoch 00003: val_loss improved from 0.44031 to 0.42474, saving model to inception_fixed_bottom_save_best.hdf5
217s - loss: 0.5810 - acc: 0.7922 - val_loss: 0.4247 - val_acc: 0.8581
Epoch 5/50
Epoch 00004: val_loss did not improve
203s - loss: 0.5900 - acc: 0.7975 - val_loss: 0.4929 - val_acc: 0.8484
Epoch 6/50
Epoch 00005: val_loss improved from 0.42474 to 0.42202, saving model to inception_fixed_bottom_save_best.hdf5
224s - loss: 0.5693 - acc: 0.8079 - val_loss: 0.4220 - val_acc: 0.8

In [ ]:
model.load_weights('inception_fixed_bottom_save_best.hdf5')

In [ ]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))